In [89]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [21]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [35]:
main_dir = 'C:\\Users\\nguye\\PycharmProjects\\DPL302\\Pose recognition'

In [38]:
label = 'Yoga'
labels_path = os.path.join(main_dir,'dataset',label)
labels = os.listdir(labels_path)

In [43]:
poses = np.array(labels)
print(poses)

['adho mukha svanasana' 'adho mukha vriksasana' 'agnistambhasana'
 'ananda balasana' 'anantasana' 'anjaneyasana' 'ardha bhekasana'
 'ardha chandrasana' 'ardha matsyendrasana' 'ardha pincha mayurasana'
 'ardha uttanasana' 'ashtanga namaskara' 'astavakrasana' 'baddha konasana'
 'bakasana' 'balasana' 'bhairavasana' 'bharadvajasana i' 'bhekasana'
 'bhujangasana' 'bhujapidasana' 'bitilasana' 'camatkarasana'
 'chakravakasana' 'chaturanga dandasana' 'dandasana' 'dhanurasana'
 'durvasasana' 'dwi pada viparita dandasana' 'eka pada koundinyanasana i'
 'eka pada koundinyanasana ii' 'eka pada rajakapotasana'
 'eka pada rajakapotasana ii' 'ganda bherundasana' 'garbha pindasana'
 'garudasana' 'gomukhasana' 'halasana' 'hanumanasana' 'janu sirsasana'
 'kapotasana' 'krounchasana' 'kurmasana' 'lolasana'
 'makara adho mukha svanasana' 'makarasana' 'malasana' 'marichyasana i'
 'marichyasana iii' 'marjaryasana' 'matsyasana' 'mayurasana'
 'natarajasana' 'padangusthasana' 'padmasana' 'parighasana'
 'paripurn

In [27]:
cap = cv.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv.imshow('OpenCV Feed', cv.flip(frame,1))

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [141]:
for pose_action in poses:
    try: 
        os.makedirs(os.path.join(main_dir, 'keypoints',pose_action))
    except:
        pass

In [142]:
img_no = 0
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    for pose_actions in poses:
        current_pose = pose_actions
        pose_path = os.path.join(main_dir,'dataset', label, pose_actions)
        pose_actions = np.array(os.listdir(pose_path))
        img_no = 0
        for img in pose_actions:
            img_path = os.path.join(main_dir,'dataset', label, current_pose, img)
            frame = cv.imread(img_path)
            
            try:
                frame.flags.writeable = False
                frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = pose.process(frame)
                frame.flags.writeable = True
                frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
            except:
                print('Error while extracting ', img, 'of', current_pose)
                img_no += 1
                pass
            
            keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
            
            kp_path = os.path.join(main_dir, 'keypoints', current_pose, str(img_no))
            np.save(kp_path, keypoints)
            img_no += 1
                
            cv.imshow('Reading', frame)
            cv.waitKey(100)


cv.destroyAllWindows()

In [144]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [145]:
label_map = {label:num for num, label in enumerate(poses)}

In [146]:
label_map

{'adho mukha svanasana': 0,
 'adho mukha vriksasana': 1,
 'agnistambhasana': 2,
 'ananda balasana': 3,
 'anantasana': 4,
 'anjaneyasana': 5,
 'ardha bhekasana': 6,
 'ardha chandrasana': 7,
 'ardha matsyendrasana': 8,
 'ardha pincha mayurasana': 9,
 'ardha uttanasana': 10,
 'ashtanga namaskara': 11,
 'astavakrasana': 12,
 'baddha konasana': 13,
 'bakasana': 14,
 'balasana': 15,
 'bhairavasana': 16,
 'bharadvajasana i': 17,
 'bhekasana': 18,
 'bhujangasana': 19,
 'bhujapidasana': 20,
 'bitilasana': 21,
 'camatkarasana': 22,
 'chakravakasana': 23,
 'chaturanga dandasana': 24,
 'dandasana': 25,
 'dhanurasana': 26,
 'durvasasana': 27,
 'dwi pada viparita dandasana': 28,
 'eka pada koundinyanasana i': 29,
 'eka pada koundinyanasana ii': 30,
 'eka pada rajakapotasana': 31,
 'eka pada rajakapotasana ii': 32,
 'ganda bherundasana': 33,
 'garbha pindasana': 34,
 'garudasana': 35,
 'gomukhasana': 36,
 'halasana': 37,
 'hanumanasana': 38,
 'janu sirsasana': 39,
 'kapotasana': 40,
 'krounchasana': 

In [170]:
poses_kp, labels_kp = [], []
for pose_action in poses:
    pose_path = os.path.join(main_dir, 'keypoints', pose_action)
    pose_count = len(os.listdir(pose_path))
    for pose_kp in range(pose_count):
        keypoints = np.load(os.path.join(main_dir, 'keypoints', pose_action,'{}.npy'.format(pose_kp)))
        poses_kp.append(keypoints)
        labels_kp.append(label_map[pose_action])
    

In [164]:
np.array(poses_kp).shape

(5991, 132)

In [173]:
np.array(labels_kp).shape

(5991,)

In [174]:
X = np.array(poses_kp)
X.shape

(5991, 132)

In [178]:
y = to_categorical(labels_kp).astype(int)
y.shape

(5991, 107)

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [189]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [190]:
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu',input_shape=(132,1)))
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu'))
model.add(MaxPooling1D(pool_size = 2))
model.add(Flatten())
model.add(Dense(128, activation= 'relu'))
model.add(Dense(107, activation= 'softmax'))

In [191]:
model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])

In [196]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 130, 64)           256       
                                                                 
 conv1d_3 (Conv1D)           (None, 128, 64)           12352     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 64, 64)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 128)               524416    
                                                                 
 dense_1 (Dense)             (None, 107)               13803     
                                                      

In [192]:
model.fit(X_train, y_train, epochs = 100, verbose = True)

Epoch 1/100
178/178 [==============================] - 4s 14ms/step - loss: 4.1112 - accuracy: 0.0958
Epoch 2/100
178/178 [==============================] - 2s 14ms/step - loss: 2.4312 - accuracy: 0.3848
Epoch 3/100
178/178 [==============================] - 2s 14ms/step - loss: 1.7845 - accuracy: 0.5271
Epoch 4/100
178/178 [==============================] - 2s 14ms/step - loss: 1.5039 - accuracy: 0.5980
Epoch 5/100
178/178 [==============================] - 2s 14ms/step - loss: 1.3459 - accuracy: 0.6371
Epoch 6/100
178/178 [==============================] - 2s 14ms/step - loss: 1.2164 - accuracy: 0.6651
Epoch 7/100
178/178 [==============================] - 2s 14ms/step - loss: 1.1355 - accuracy: 0.6848
Epoch 8/100
178/178 [==============================] - 2s 14ms/step - loss: 1.0672 - accuracy: 0.7053
Epoch 9/100
178/178 [==============================] - 2s 14ms/step - loss: 0.9888 - accuracy: 0.7259
Epoch 10/100
178/178 [==============================] - 2s 14ms/step - loss: 0.938

178/178 [==============================] - 3s 16ms/step - loss: 0.3898 - accuracy: 0.8981
Epoch 81/100
178/178 [==============================] - 3s 16ms/step - loss: 0.4085 - accuracy: 0.8940
Epoch 82/100
178/178 [==============================] - 3s 16ms/step - loss: 0.4178 - accuracy: 0.8875
Epoch 83/100
178/178 [==============================] - 3s 16ms/step - loss: 0.4220 - accuracy: 0.8912
Epoch 84/100
178/178 [==============================] - 3s 16ms/step - loss: 0.3999 - accuracy: 0.8958
Epoch 85/100
178/178 [==============================] - 3s 16ms/step - loss: 0.3966 - accuracy: 0.8963
Epoch 86/100
178/178 [==============================] - 3s 16ms/step - loss: 0.3946 - accuracy: 0.8967
Epoch 87/100
178/178 [==============================] - 3s 16ms/step - loss: 0.4021 - accuracy: 0.8940
Epoch 88/100
178/178 [==============================] - 3s 16ms/step - loss: 0.3972 - accuracy: 0.8977
Epoch 89/100
178/178 [==============================] - 3s 16ms/step - loss: 0.4000 - 

In [231]:
pred = model.predict(X_test)
pred_y = pred.argmax(axis = -1)
print(pred_y)

10/10 [==============================] - 0s 5ms/step
[ 85  85  75  12  73  14   0  21  62  80  30  76  56  94  43  34  89  48
  69  82  95  28   3  79   0  45  97  59  74  66  70  61  85  71  39  35
  29  85  71  96  91  17  34  10  78   2  31  90   5   0   6   5  99  52
   0  34  36  20  58  64  65  58  15  85  63  52   7  90  98  76  10  66
  19   0  85   4  22  94  86  97  19  10  94  52  52  60  14  62  13  21
  45  10   3  42  63  85  49  26  85  17 104 104  61  23  69  69  31  47
  17  21  52  21  47  99  21  46 104 104  85  92 105  85  57  37  60 105
  52  51  18  27  81  97  17  30  19  70  56  40  58  84  52  98  76 106
  78  63  13 102  85  49  77  40  39   0 101  81  61  45  85  94 102  62
  60  11  63  96  30  87  82  61  51  21  78  13   0  24  63  90  82  88
  64  14  87 102   3  87  37 104  35  17 101  35  19  93  44  72  63  60
  74  32   9  25  56  85  31  75   9   8  13   6  96  75  93  85  79  85
  17   5  85  42  78 100  80  23  11  40  66   0  12  14 100  47  84  9

In [221]:
image = 'C:\\Users\\nguye\\PycharmProjects\\DPL302\\Pose recognition\\dataset\\Yoga\\dandasana\\28-0.png'
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    while True:
        frame = cv.imread(image)

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

        keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv.imshow('kp',frame)
        
        if cv.waitKey(5) & 0xFF == ord('q'):
            break

cv.destroyAllWindows()



InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/conv1d_2/Conv1D/ExpandDims' defined at (most recent call last):
    File "C:\Users\nguye\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\nguye\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    

In [ ]:
img_no = 0
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    for pose_actions in poses:
        current_pose = pose_actions
        pose_path = os.path.join(main_dir,'dataset', label, pose_actions)
        pose_actions = np.array(os.listdir(pose_path))
        img_no = 0
        for img in pose_actions:
            img_path = os.path.join(main_dir,'dataset', label, current_pose, img)
            frame = cv.imread(img_path)
            
            try:
                frame.flags.writeable = False
                frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = pose.process(frame)
                frame.flags.writeable = True
                frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
            except:
                print('Error while extracting ', img, 'of', current_pose)
                img_no += 1
                pass
            
            keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
            
            kp_path = os.path.join(main_dir, 'keypoints', current_pose, str(img_no))
            np.save(kp_path, keypoints)
            img_no += 1
                
            cv.imshow('Reading', frame)
            cv.waitKey(100)


cv.destroyAllWindows()